In [2]:
import pandas as pd, json, matplotlib.pyplot as plt
%matplotlib inline

In [3]:
ls *.csv

data_by_sphere.csv  scenario.csv


In [4]:
pd.read_csv('data_by_sphere.csv')

,source,is_vde,sphere,year,k_t_ne
0,Газ,0,total,2012,21698.0
1,Електроенергія,0,total,2012,11130.0
2,Теплова енергія,0,total,2012,11853.0
3,Біопаливо та відходи,1,total,2012,1029.0
4,Вугілля,0,total,2012,9212.0
5,Нафта та нафтопродукти,0,total,2012,11347.0
6,Сонячна енергія,1,total,2012,0.0
7,Теплова енергія з ВДЕ,1,total,2012,12.3
8,Електроенергія з ВДЕ,1,total,2012,710.0
9,Газ,0,total,2035,7615.0


In [ ]:
{
    'ДОМОГОСПОДАРСТВА': [
        'Термомодернізація житлових будинків',
        'Модернізація систем енергозабезпечення об’єктів ЖКГ',
        'Автоматизовані індивідуальні теплові пункти (ІТП)',
        'Впровадження когенераційних систем для багатоповерхових будинків',
        'Техніка А++, А+++',
        '40-50% населення використовує сонячні панелі на дахах будинків',
        'Опалення та гаряча вода за рахунок біопалива та відходів, централізованого ВДЕ-тепла та використання сонячних колекторів',
    ],
    'ТРАНСПОРТ': [
        'Легкові авто: 90% електромобілі, 10% на біопаливі;',
        'Електроавтобуси та електропотяги;',
        'Авіаційний, водний, вантажний транспорт на біопаливі.',
    ],
    'ПРОМИСЛОВІСТЬ': [
        '88% - енергія з відновлюваних джерел та альтернативних видів палива;',
        'Електрифікація промисловості;',
        'Заміна енергоємного обладнання на нове та ефективне;',
        'Більші інвестиції з боку промислових виробників, зокрема електроенергетичної галузі.',
    ],
    ''
}